In [1]:
import pandas as pd
from bertopic import BERTopic
import plotly.express as px
from umap import UMAP
from bertopic.representation import KeyBERTInspired
import numpy as np 
from sentence_transformers import SentenceTransformer

/Users/martynagrygiel/Documents/studia/sem2/amc/wiatrolapy/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('max_colwidth', None)

# Analiza zaanotowanych wypowiedzi z Sejmu przez BERTopic

Ten notebook zawiera następujące klastry wydzielone przez bertopic:
* 2
* 7
* 11
* 18
* 23
* 35
* 38
* 41
* 50
* 56
* 57
* 61
* 76
* 79
* 90


Wydzielenie klastrów zostało przygotowane w notebooku **bertopic_sejm_embedding.ipynb**

In [ ]:
sejm = pd.read_csv("data/bertopic_results/sejm/sejm_bertopic_annotated.csv")
sejm['topic_name'] = ""
# sejm = pd.read_csv("data/sejm_bertopic_updated.csv")
sejm

## Topic 0

In [5]:
sejm_t0 = sejm[sejm['detected_topic'] == 0]
sejm_t0.reset_index(inplace=True)

In [6]:
num_partitions = 5
partition_size = len(sejm_t0) // num_partitions

# Create a list to store the 5 DataFrames
split_dataframes = []

# Loop through and create the 5 DataFrames
for i in range(num_partitions):
    start_index = i * partition_size
    end_index = (i + 1) * partition_size if i < num_partitions - 1 else None
    
    # Use iloc to get the rows for the current partition
    partition_df = sejm_t0.iloc[start_index:end_index]
    
    # Append the partition DataFrame to the list
    split_dataframes.append(partition_df)

In [53]:
embedding_model='paraphrase-multilingual-MiniLM-L12-v2'
sentence_model = SentenceTransformer(embedding_model, device='mps')
embeddings: np.ndarray = sentence_model.encode(split_dataframes[4].text_all.to_numpy(), show_progress_bar=True, batch_size=128)
representation_model = KeyBERTInspired(random_state=42)
calculate_probabilities = True # get document's topic probs for each detected topic, not just the predicted one
umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine', random_state=42, verbose=True)

nr_topics: int | str = 15
topic_model = BERTopic(embedding_model=embedding_model, nr_topics=nr_topics, verbose=True, representation_model=representation_model, calculate_probabilities=calculate_probabilities, umap_model=umap_model, low_memory=True)
topics, probs = topic_model.fit_transform(split_dataframes[4].text_all, embeddings=embeddings)

Batches: 100%|██████████| 492/492 [04:03<00:00,  2.02it/s]


UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_neighbors=30, random_state=42, verbose=True)
Wed Jan  3 20:33:50 2024 Construct fuzzy simplicial set
Wed Jan  3 20:33:50 2024 Finding Nearest Neighbors
Wed Jan  3 20:33:50 2024 Building RP forest with 18 trees
Wed Jan  3 20:33:51 2024 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	 5  /  16
	Stopping threshold met -- exiting after 5 iterations
Wed Jan  3 20:33:57 2024 Finished Nearest Neighbor Search
Wed Jan  3 20:33:57 2024 Construct embedding


Epochs completed:   2%| ▏          3/200 [00:01]

	completed  0  /  200 epochs


Epochs completed:  11%| █          22/200 [00:04]

...

Epochs completed:  91%| █████████  182/200 [00:38]

	completed  180  /  200 epochs


Epochs completed: 100%| ██████████ 200/200 [00:42]


Wed Jan  3 20:34:48 2024 Finished embedding


2024-01-03 20:34:48,452 - BERTopic - Reduced dimensionality
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENI

In [54]:
topic_model.visualize_barchart(top_n_topics=nr_topics if isinstance(nr_topics, int) else 50, n_words=20)

In [55]:
ignore_outlier = True
topic_info_df = topic_model.get_topic_info()
if (ignore_outlier):
    topic_info_df = topic_info_df.loc[topic_info_df.Topic != -1]

fig = px.histogram(y=topic_info_df['Name'], x=topic_info_df['Count'], orientation='h', title='Histogram of topic occurences' + (' (outlier topic -1 omitted)' if ignore_outlier else ''))
fig.update_layout(height=900)
fig.update_xaxes(title_text='# of Occurences')

# Update y-axis label
fig.update_yaxes(title_text='Topic name')
fig.show()

In [56]:
new_topics = topic_model.reduce_outliers(split_dataframes[4].text_all, topics, probabilities=probs, strategy='probabilities')

In [57]:
split_dataframes[4]['detected_topic'] = new_topics

In [58]:
topic_mapping = {0: '', 
                 1: 'publiczna opieka zdrowotna', 
                 2: 'wojsko',
                 3: '',
                 4: 'emerytura',
                 5: 'infrastruktura drogowa',
                 6: 'rybołóstwo',
                 7: 'niepełnosprawni',
                 8: '',
                 9: '',
                 10: '',
                 11: '',
                 12: '',
                 13: '',
}
split_dataframes[4]['topic_name'] = split_dataframes[4]['detected_topic'].map(topic_mapping)

In [59]:
split_dataframes[4]['topic_name'].unique()

array(['', 'emerytura', 'wojsko', 'publiczna opieka zdrowotna',
       'niepełnosprawni', 'infrastruktura drogowa', 'rybołóstwo'],
      dtype=object)

In [60]:
split_dataframes[4].to_csv("data/sejm_bertopic_updated_topic0_part4.csv")

In [61]:
dfs = []
for i in range(5):
    file_name = f'data/sejm_bertopic_updated_topic0_part{i}.csv'
    df = pd.read_csv(file_name)
    dfs.append(df)

# Concatenate the DataFrames
result_df = pd.concat(dfs, ignore_index=True)
result_df

Unnamed: 0    index        date                     author  \
0                0       28  2019-11-12  senior antoni macierewicz   
1                1       29  2019-11-12                borys budka   
2                2       42  2019-11-12  władysław kosiniak-kamysz   
3                3       53  2019-11-12            robert winnicki   
4                4       60  2019-11-12  senior antoni macierewicz   
...            ...      ...         ...                        ...   
314506      314506  1763474  2020-02-06               jerzy wcisła   
314507      314507  1763476  2020-02-06             krzysztof słoń   
314508      314508  1763480  2020-02-06           jerzy czerwiński   
314509      314509  1763482  2020-02-06           jerzy czerwiński   
314510      314510  1763484  2020-02-06             marcin bosacki   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [63]:
result_df['topic_name'] = result_df['topic_name'].replace('odnawialne źródła energii', 'energia odnawialna')

In [64]:
result_df['topic_name'].unique()

array([nan, 'gospodarka odpadowa', 'publiczna opieka zdrowotna',
       'emerytura', 'węgiel', 'prawa zwierząt', 'kościół', 'aborcja',
       'energetyka jądrowa', 'lotnictwo', 'rolnictwo', 'lasy państwowe',
       'prawo antyterrorystyczne', 'energia odnawialna', 'imigranci',
       'podatki', 'niepełnosprawni', 'policja', 'szkolnictwo', 'inflacja',
       'wojsko', 'infrastruktura drogowa', 'rybołóstwo'], dtype=object)

In [65]:
result_df.to_csv("data/sejm_topic_0.csv")

## Topic 90

In [16]:
sejm_t90 = sejm[sejm['detected_topic'] == 90]
sejm_t90[['text_all', 'topic_name']]

,text_all,topic_name
549930,"panie marszałku! panie ministrze! wysoka izbo! w związku z omawianą ustawą o zmianie ustawy prawo budowlane oraz niektórych innych ustaw chciałbym prosić o wyjaśnienie kilku spraw. pierwsze moje pytanie dotyczy art. 29 ustawy prawo budowlane, konkretnie zapisu zawartego w ust. 2 pkt 17, mówiącego o tym, że zgłoszenia zamiaru wykonania robót budowlanych nie wymagają roboty budowlane polegające m.in. na remoncie instalacji wewnętrznych w obrębie budynku. prosiłbym o wyjaśnienie, czy pod tym pojęciem należy rozumieć również roboty polegające np. na zmianie sposobu ogrzewania z piecowego na centralne ogrzewanie i związanym z tym wykonaniem instalacji cieplnej w cały budynku. drugie moje pytanie jest związane z uchyleniem art. 65 ustawy o planowaniu i zagospodarowaniu przestrzennym. przepis ten mówi o przesłankach wygaśnięcia decyzji o warunkach zabudowy oraz decyzji o ustaleniu lokalizacji celu publicznego. jedną z tych przesłanek jest uchwalenie miejscowego planu zagospodarowania przestrzennego, którego ustalenia są inne niż w wydanej decyzji. prosiłbym o wyjaśnienie, dlaczego ten przepis jest uchylany. dziękuję bardzo.",prawo budowlane
570526,"dziękuję bardzo. panie marszałku! wysoka izbo! do tej pory samowole budowlane były traktowane w sposób jasny, wyraźny i zgodny z jednoznacznymi zapisami prawa budowlanego. naliczane były kary za samowolę budowlaną, a w przypadku gdy samowola była realizowana w obszarze dopuszczającym realizację przedsięwzięć tego typu w planie zagospodarowania przestrzennego, można było legalizować samowolę za specjalną opłatą. naliczanie tej opłaty było i jest do tej pory określone w obowiązującym prawie. odpowiednie zapisy są zawarte w art. 49 i 59f. w art. 55 określono kategorie, natomiast w proponowanym przez wnioskodawców art. 49 wpisano kwotę 5 tys. zł bez żadnych dodatkowych odesłań do art. 29. jaką to powoduje sytuację? kieruję pytanie do pani poseł sprawozdawcy. czy w związku z tymi niejasnymi, zagmatwanymi zapisami nie będzie się naliczało tylko jednej opłaty legalizacyjnej w wysokości 5 tys. zł w odniesieniu i do garażu, i do pięćdziesięciopiętrowego wieżowca, i do całego osiedla mieszkaniowego, realizowanego na mocy jednej decyzji o pozwoleniu na budowę czy też jednej rejestracji? specjaliści, eksperci mówią, że kiedy przepisy te wejdą w życie, będzie się opłacało realizować samowole budowlane, ponieważ koszt legalizacji będzie mniejszy aniżeli koszt projektu budowlanego w przypadku dużych realizacji. dziękuję bardzo.",prawo budowlane
768079,"panie marszałku! wysoka izbo! projektowana ustawa doprecyzowuje w art. 30 ust. 5e problematykę ostemplowania projektu budowlanego objętego zgłoszeniem budowy lub przebudowy budynku mieszkalnego jednorodzinnego. trzeba przy tym podkreślić, że ostemplowanie jest czynnością techniczną potrzebną do celów dowodowych. z tą czynnością nie wiążą się szczególne prawa ani obowiązki po stronie inwestora, ma ona bowiem jedynie charakter dowodowy. organ nadzoru budowlanego, prowadząc kontrolę wykonywania robót budowlanych zgodnie z projektem budowlanym, posługuje się projektem budowlanym znajdującym się na budowie. ostemplowanie zaś zapewni, że znajdujący się na budowie projekt budowlany, którym posługują się uczestniczy procesu budowlanego, jest tym projektem budowlanym, który był przedmiotem zgłoszenia. w związku z tym czy braki w ostemplowaniu projektu będą traktowane jako wada istotna w procesie jego zgłoszenia i czy mogą spowodować przedłużenie terminu wydania decyzji o pozwoleniu na budowę? dziękuję bardzo.",prawo budowlane
1272782,"szanowni państwo, poprawka, którą, jak mówiłam, złożę w drugim czytaniu, odnosi się do tego, żeby wykreślić możliwość projektowania w specjalności architektonicznej w ograniczonym zakresie technikum lub szkoły branżowej, czyli zawodówki ii stopnia. chciałam zwrócić uwagę, że architekturę studiuje… oprócz dyscyplin, które są oczywiste, czyli trzy lata budownictwa, konstrukcji budowlanych, potem mamy materia

In [15]:
sejm_t90['topic_name'] = 'prawo budowlane'
sejm.loc[sejm.index.isin(sejm_t90.index), 'topic_name'] = sejm_t90['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/372255947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t90['topic_name'] = 'prawo budowlane'


## Topic 79

In [17]:
sejm_t79 = sejm[sejm['detected_topic'] == 79]
sejm_t79[['text_all', 'topic_name']]

,text_all,topic_name
684574,"panie marszałku! wysoki sejmie! afera amber gold zatacza coraz szersze kręgi, ale wsłuchując się w przekaz niektórych mediów, nie sposób nie odnieść wrażenia, że ich zdaniem najbardziej godnym uwagi aspektem tej sprawy jest trzymilionowa dziura w budżecie filmu o lechu wałęsie. w związku z tym niektórzy dziennikarze, politycy i ludzie związani z kulturą zdają się prześcigać w ubolewaniu nad tym faktem, a co za tym idzie nad opóźnieniem w stworzeniu tego rzekomo wiekopomnego dzieła. jednak ich rozpaczliwy ton jakoś specjalnie nie robi na mnie wrażenia, ponieważ jestem przekonany, że obecni decydenci zrobią wszystko, aby ten film został rychło ukończony, w odróżnieniu np. od produkcji filmowej o żołnierzu wyklętym - sierżancie dziemieszkiewiczu, której autorzy po zmianie władzy w telewizji zostali zmuszeni do zbierania pieniędzy na ukończenie filmu podczas organizowanych w całej polsce zbiórek publicznych. ponadto widzę, jak ostatnio, w czasie rządów platformy obywatelskiej, skutecznie zablokowano wiele cennych dla naszej historii i kultury inicjatyw. w moim rodzinnym tarnowie ma zostać wydane opracowanie naukowe poświęcone represjom komunistycznym na terenie powiatu tarnowskiego. niestety wobec obcięcia budżetu i tym samym braku dostatecznych środków na ten cel instytut pamięci narodowej musi szukać sponsorów. ponadto od wielu dni z uwagą śledzę informacje na temat prac archeologicznych na terenie cmentarza powązkowskiego, gdzie na tzw. łączce wydobywane są ciała ofiar komunistycznych zbrodni. tak jak wielu polaków czekam na ostateczne wyniki prac archeologów, genetyków i historyków z ipn, które być może pozwolą ustalić nazwiska ofiar, a także będą bezcennym materiałem oskarżającym zbrodniarzy aparatu komunistycznego. niestety w międzyczasie dowiedzieliśmy się, że ipn wkrótce może zostać pozbawiony swojej głównej siedziby w warszawie i wiele wskazuje na to, że ministerstwo skarbu państwa może nie przygotować dla tej ważnej instytucji nowego budynku. a więc z jednej strony niby robi się wszystko, żeby wesprzeć działania instytutu, a z drugiej nagle spółka ruch sprzedaje swój budynek i tym samym instytut może nie tylko stracić swoją siedzibę, lecz co gorsza wkrótce jego statutowa działalność może stanąć pod znakiem zapytania. czy to przypadek? w obecnych realiach szczerze wątpię i bardzo nad tym ubolewam, że przyszło nam żyć pod rządami tych, którzy najwyraźniej za bardzo faworyzują niektórych styropianowych bohaterów, a pamięci o tych autentycznych chyba najzwyczajniej się obawiają.",
684717,"pani marszałek! wysoka izbo! w związku z dzisiejszą informacją pana premiera wskazującą, że kierowana przez marcina p. spółka amber gold od wielu lat prowadziła działalność niezgodną z prawem bankowym i w efekcie dopuściła się wielomilionowych wyłudzeń, oraz mając na uwadze ustawowy obowiązek zwalczania tego rodzaju działalności przestępczej przez podległe panu służby specjalne, proszę o odpowiedź na następujące pytania. pierwsze pytanie: czy podległe panu służby sporządzały jakiekolwiek opracowania na bazie uzyskiwanych informacji, niezależnie od tego, jaki był rodzaj źródła tych informacji, o działalności spółki amber gold? jeśli tak, to kiedy? proszę o podanie daty wytworzenia dokumentów oraz kto był zapoznany z treścią tych dokumentów? pytanie drugie: czy podległe panu służby sporządzały jakiekolwiek opracowania dotyczące działalności biznesowej marcina p.? jeśli tak, to kiedy? proszę podać dokładną datę wytworzenia dokumentów oraz kto był zapoznany z treścią tych dokumentów? pani marszałek, jeszcze dwa pytania. czy w zasobach informacji zgromadzonych w służbach specjalnych znajdują się dane dotyczące marcina p. i spółki amber gold? od kiedy? proszę podać datę wprowadzenia do bazy danych oraz która jednostka wprowadziła te informacje. pytanie ostatnie: czy i kiedy były dokonywane sprawdzenia dotyczące marcina p. i spółki amber gold w niejawnych zasobach służb specjalnych, na przykład w zasobie cezop? panie p

Tutaj częściowo odnoszą się do samego amber gold, częściowo do obecnych rządów, przedsiębiorców dlatego overall do wyrzucenia

## Topic 76

In [20]:
sejm_t76 = sejm[sejm['detected_topic'] == 76]


In [35]:
sejm_t76[['text_all', 'topic_name']]

text_all  \
49703                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [36]:
sejm_t76['topic_name'] = 'pomoc dla Ukrainy'
sejm.loc[sejm.index.isin(sejm_t76.index), 'topic_name'] = sejm_t76['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/3511657890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t76['topic_name'] = 'pomoc dla Ukrainy'


## Topic 61

In [37]:
sejm_t61 = sejm[sejm['detected_topic'] == 61]

In [39]:
sejm_t61[['text_all', 'topic_name']].iloc[:15]

,text_all,topic_name
20329,"dziękuję. pani marszałek! wysoka izbo! będziemy składać wiele poprawek. to będą takie poprawki, test na tzw. dobrego człowieka, o którym mówił prezes kaczyński. jedna z poprawek mówi o zakazie wykorzystywania koni na drodze do morskiego oka. chyba już najwyższy czas, aby to barbarzyństwo się zakończyło, aby na naszych oczach po prostu nie ginęły konie. to chyba jedyny, swoją drogą, park narodowy, gdzie na naszych oczach giną konie. w ogóle nie wiesz, o czym mówisz. precz z wyścigami na służewcu. to jest taki test na dobrego człowieka. to pytanie zadaję prezesowi kaczyńskiemu, czy chce być dobry, jak mówi, i poprze projekt ochrony zwierząt akurat w przypadku tych koni, czy też potraktuje to wybiórczo i będzie popierał tatrzański park narodowy, który w oświadczeniu i w odpowiedzi twierdzi, że nie wie, o co w ogóle chodzi. dziękuję bardzo. koniec z kanarkami w klatkach. wypuścić kanarki.",
20815,"dziękuję. panie marszałku! wysoka izbo! będę konsekwentnie przekonywał wszystkich na tej sali, aby wprowadzić zakaz wykorzystywania koni na drodze do morskiego oka. dlaczego nie na innych drogach? otóż już w latach 90. przewodnicy tatrzańscy, ratownicy topr informowali o padających koniach i mówili, że trzeba coś z tym zrobić. wszyscy widzieliśmy zdjęcia, filmy. myślę, że na swój sposób każdy jest wrażliwy, i wydaje mi się, że nie muszę dalej przekonywać. z panią poseł piekarską składaliśmy interpelację w czerwcu w tej sprawie. do dzisiaj zresztą nie ma odpowiedzi. ale myślę, że to może nas połączyć. jeśli myślicie o tym, żeby ochronić zwierzęta, to możemy właśnie coś dobrego dla tych koni zrobić. jest taki utwór maryli rodowicz, tekst do niego napisała agnieszka osiecka „tylko koni, tylko koni, tylko koni żal”. wam nie żal? nie żal wam, jak konie zdychają, umierają w męczarniach? apeluję do was, abyście to poparli, bo możemy wspólnie coś dobrego zrobić. dziękuję.",
62802,"panie marszałku! wysoka izbo! koń jaki jest, każdy widzi - to jest bardzo dobra definicja konia z ˝nowych aten˝ ks. benedykta chmielowskiego. i tak samo: kto to jest dobry analityk, to też wiadomo. tego nie można ustalić przez głosowanie, nie może tego ustalić sejm. to wiedzą tylko praktycy. jak głosowałem nad 163, o ile pamiętam, artykułami tej ustawy, tak sobie myślałem: potrzebna jest jedna ustawa, która by mówiła, że analityk działa dobrze w granicach swoich możliwości i umiejętności. i tyle. tutaj poseł, chyba z pis-u, chwalił się, że tam jest taki paragraf, który zobowiązuje analityków do pogłębiania swojej wiedzy. a może trzeba, żeby tam było, że ma poszerzać swoją wiedzę, a nie pogłębiać, albo jakoś podobnie. przecież to jest śmieszne, proszę państwa. to wszystko jest śmieszne. ktoś może kończyć wydział nie biologii tylko nanobiologii, bo może na tym uniwerku nie ma wydziału biologii tylko jest nanobiologii albo mikrobiologii. i co, to jest zły kierunek? przepraszam bardzo. no oczywiście. są tysiące rozmaitych kierunków, które powstają, i zapewniam państwa, że loty kosmiczne wykonywali ludzie, którzy nie kończyli wydziału lotów kosmicznych. tak że, proszę łaskawie, my się wstrzymamy od głosu nad tą ustawą, ponieważ ona jest tak samo głupia jak poprzednia, więc nie ma to żadnego znaczenia. poprawia kilka drobiazgów, więc my nie chcemy głosować przeciwko tej ustawie, bo ona nic nam nie szkodzi. ale, proszę państwa, pozbądźmy się myśli, że sejm może wszystko uregulować. życie jest znacznie ważniejsze niż formułki prawne. kolega braun jeszcze chce bardzo ważne sprawy dodać na końcu. dziękuję bardzo.",
579993,"panie marszałku! panie ministrze! moje pytania dotyczą znakowania koniowatych. pytanie pierwsze: dlaczego wprowadza się w przypadku koni podwójny system znakowania: dotychczasowy opis słowno-graficzny i drugi, nowy, czipy? drugie pytanie: czy w krajach unii europejskiej również prowadzone są dwa systemy znakowania koni, o których wspomniałem w pierwszym pytaniu? po trzecie: jakie będą koszty znakowania i opisu jednego konia, które p

In [40]:
sejm_t61['topic_name'] = 'wyścigi konne'
sejm.loc[sejm.index.isin(sejm_t61.index), 'topic_name'] = sejm_t61['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/646769818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t61['topic_name'] = 'wyścigi konne'


## Topic 57

In [42]:
sejm_t57 = sejm[sejm['detected_topic'] == 57]
random_sample = sejm_t57.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
36449                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [43]:
sejm_t57['topic_name'] = 'zakaz handlu w niedzielę'
sejm.loc[sejm.index.isin(sejm_t57.index), 'topic_name'] = sejm_t57['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/1690025185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t57['topic_name'] = 'zakaz handlu w niedzielę'


## Topic 56

In [44]:
sejm_t56 = sejm[sejm['detected_topic'] == 56]
random_sample = sejm_t56.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
831327                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       panie marszałku! pani premier! wysoka izbo! dzisiaj platforma obywatelska proponuje odrzucenie projektu ustawy antyterrorystycznej w ca

In [45]:
sejm_t56['topic_name'] = 'prawo antyterrorystyczne'
sejm.loc[sejm.index.isin(sejm_t56.index), 'topic_name'] = sejm_t56['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/228327525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t56['topic_name'] = 'prawo antyterrorystyczne'


## Topic 50 

In [46]:
sejm_t50 = sejm[sejm['detected_topic'] == 50]
random_sample = sejm_t50.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
556381                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [47]:
sejm_t50['topic_name'] = 'kontrola PIP i sanepidu'
sejm.loc[sejm.index.isin(sejm_t50.index), 'topic_name'] = sejm_t50['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/3884155466.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t50['topic_name'] = 'kontrola PIP i sanepidu'


## Topic 41

In [48]:
sejm_t41 = sejm[sejm['detected_topic'] == 41]
random_sample = sejm_t41.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

,text_all,topic_name
591313,"panie marszałku! wysoka izbo! jednym z celów w zaproponowanej przez klub parlamentarny prawo i sprawiedliwość nowelizacji jest wzięcie w obronę tych najbardziej bezbronnych, dzieci, które często są ofiarami przestępstw seksualnych. wiem, jak bardzo odrażające są to przestępstwa, pozostawiają one trwały ślad w psychice człowieka. państwo powinno nie tylko karać sprawców przestępstw seksualnych, ale powinno też zabezpieczać małoletnich przed możliwością ponownego popełnienia przestępstwa przez te osoby. jedną z propozycji, która była zawarta w tej nowelizacji, było podawanie do publicznej wiadomości nazwisk osób, które takich czynów się dopuściły. jest to jedno z rozwiązań, o którym mówi się w stanach zjednoczonych i w europie. powstają listy z nazwiskami pedofilów, które albo są powszechnie dostępne, albo służą organom państwa i mają im pomóc w sprawowaniu kontroli. ta propozycja nie spotkała się z przychylnością komisji. stąd moje pytanie do ministra sprawiedliwości: czy ministerstwo sprawiedliwości tworzy takie listy i czy osoby, które popełniały takie przestępstwa, są w jakiś sposób objęte szczególnym nadzorem ze strony organów państwa?",
945565,"szanowni państwo! kandydat pis na rzecznika nie spełnia kryteriów wyboru. to jawne łamanie prawa. w czasie, gdy tak mocno piętnowana jest pedofilia w kościele, prawo i sprawiedliwość stawia na kandydata blisko związanego z kościołem. co chcecie tą kandydaturą udowodnić? że macie tak naprawdę gdzieś prawa dziecka? moje pytanie dzisiaj do pana, bo nie odpowiadał pan na pytania czy raczej wybiórczo odpowiadał na pytania w komisji, za to odpowiedział pan na wszystkie pytania ordo iuris: czy w ramach swoich działań podejmie się pan prace nad narodową strategią walki z przemocą? jaka jest pana opinia odnośnie do kodeksu rodzinnego oraz prawa rodzinnego? jakie pilne zmiany pan widzi na dzisiaj i jaki będzie harmonogram ich wdrożeń? jak będzie pan prowadził działania w zakresie monitoringu skutków deformy edukacyjnej? w komisji nie chciał pan odpowiedzieć na pytania, jaka jest pana opinia dotycząca in vitro. dlaczego? szanowni państwo! nelson mandela powiedział: naród powinien być osądzany przez to, jak traktuje swoich najsłabszych. a wy do obrony najsłabszych wystawiliście najsłabszego kandydata.",
758552,"pytania, panie niesiołowski. mam trzy pytania do przedstawiciela inicjatywy ustawodawczej. pierwsze: na jakiej podstawie twierdzi pan, że edukatorzy seksualni uczą dzieci masturbacji? pytanie drugie: jakie środowiska krytykują i oprotestowują waszą inicjatywę? i proszę wyjaśnić wysokiej izbie, szczególnie posłom platformy obywatelskiej, szczególnie panu niesiołowskiemu, który kpi , co to jest pozytywna pedofilia. dziękuję bardzo.",
960558,"gdyby rzeczywiście kierowała wami jakakolwiek troska, to byłaby reakcja ze strony prawa i sprawiedliwości, kiedy np. saryusz-wolski powiedział, że pedofilia w kościele to problem wydumany , byłaby reakcja z waszej strony na to, co powiedział terlecki. terlecki powiedział, że przecież jest to tylko i wyłącznie film atakujący kościół.",
972152,"te osoby mogą mieć albo ułatwioną drogę do popełnienia przestępstw, albo maksymalnie utrudnioną. logika kodeksu karnego, który już obowiązuje, nakazuje konsekwentne uszczelnienie granicy, która nie była wystarczająco szczelnie postawiona w 2009 r. jeśli chodzi o samych drapieżców, którzy mają trwałą skłonność do przestępstw na tle seksualnym przeciwko dzieciom... przedstawienia seksualne czy mowa o dzieciach w kontekście seksualnym stanowi dla nich bodziec wyzwalający czyny zakazane. popkultura, która rozwija się w niebezpiecznym tempie, na co panie i panowie posłowie zwracaliście uwagę, w stronę przekraczania granic, w stronę pornografizowania coraz szerszych sfer życia społecznego, coraz częściej wykorzystuje dzieci jako obiekty. granice kulturowe są przesuwane, jeżeli na ich straży nikt nie stoi w sposób stanowczy. ta kultura nie jest po to, żeby osobom dorosłym żyło się komfortowo 

In [49]:
sejm_t41['topic_name'] = 'zaostrzenie przepisów dotyczących pedofilii'
sejm.loc[sejm.index.isin(sejm_t41.index), 'topic_name'] = sejm_t41['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/181444528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t41['topic_name'] = 'zaostrzenie przepisów dotyczących pedofilii'


## Topic 35

In [51]:
sejm_t35 = sejm[sejm['detected_topic'] == 35]
random_sample = sejm_t35.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
524281                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          panie marszałku! wysoka izbo! w mroźny niedzielny dzień 17 lutego 2008 r. stanąłem pod kurhanem 360 zamordowanych skarżyszczan, by wraz z licznie zgromadzonymi mieszkańcami miasta odmówić modlitwę za dusze ofiar niemieckiego barbarzyństwa. zbrodnia, którą w związku z jej 68. rocznicą pragnę przywołać, była pierwszą wielką egzekucją na kielecczyźnie i jedną z największych w okresie hitlerowskiej okupacji. w dniu 28 stycznia 1940 r. przybył do skarżyska oddział ss i zajął kwaterę w szkole nr 1 przy ul. konarskiego. od 29 stycznia do 10 lutego 1940 r. hitlerowcy przeprowadzili w moim mieście masowe aresztowania. dotknęły one ok. 420 osób. wśród aresztowanych było wielu robotników fabryki amunicji, kolejarzy, liczni przedstawiciele inteligencji i przedsiębiorców. aresztowano 7 zakonników franciszkanów, a także burmistrza i jego zastępców. zwraca uwagę fakt, że wśród aresztowanych znalazła się nieduża grupa członków i działaczy różnych organizacji społecznych. byli to przede wszystkim harcerze w liczbie 70 osób, na czele ze swoimi komendantami: jadwigą barańską, józefem małowiczem, wandą wysocką, oraz działacze ˝strzelca˝ i ˝sokoła˝. aresztowanych przetrzymywano w budynku szkoły nr 1, a także w piwnicach magistratu miejskiego przy obecnej ul. 1 maja. w czasie przesłuchań wobec więźniów stosowano tortury, w wyniku których dwóch aresztowanych zmarło, a dwoje popełniło samobójstwo. dnia 2 lutego 1940 r. w jednym z pomieszczeń szkoły rozpoczął urzędowanie sąd policyjny. nietrwające dłużej niż 15 minut procesy kończyły się wydaniem wyroku śmierci. od świtu 12 lutego 1940 r. esesmani zaczęli skazanych wywozić i rozstrzeliwać. odbywało się to w ten sposób, że pod wejściowe drzwi szkoły podjeżdżały tyłem z opuszczoną klapą ciężarówki. skazanych grupami od 5 do 10 osób, często bijąc, załadowywano na samochody, a następnie wywożono do sosnowego losu w osiedlu bór przy drodze skarżysko-końskie. skazańców wywożono przez 4 zimowe dni: 12, 13, 14 i 15 lutego 1940 r. wykonano wyroki na 360 osobach, w tym stracono wszystkich aresztowanych harcerzy. ciała zabitych wpadały lub były wrzucane do uprzednio wykopanych dołów. pomordowani z uwagi na panujący wówczas mróz nie byli grzebani aż do ostatniego dnia egzekucji. 60 osób aresztowanych, a nie straconych na borze zostało wywiezionych do radomia i tam w przeważającej większości zamordowanych na firleju jak burmistrz franciszek tatkowski i jego zastępca maciejewski lub wysłanych do obozów koncentracyjnych. zbrodnia na mieszkańcach skarżyska-kamiennej poraża swoją bezwzględnością i okrucieństwem. unicestwiono środowisko harcerskie miasta skarżyska. nie wahano się wymordować nawet całych rodzin jak rodziny wątłych i gontaszewskich. nie szczędzono nieletnich mających 15, 16, 17 lat życia. zamordowano wszystkich ojców franciszkanów 

Temat do wyrzucenia

## Topic 23

In [52]:
sejm_t23 = sejm[sejm['detected_topic'] == 23]
random_sample = sejm_t23.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

,text_all,topic_name
1696319,"czy ktoś z pań i panów senatorów chce zabrać głos w sprawie porządku obrad? nie widzę zgłoszeń. dziękuję bardzo. stwierdzam, że senat zatwierdził porządek obrad trzydziestego trzeciego posiedzenia senatu rzeczypospolitej polskiej ix kadencji. informuje państwa senatorów, że dziś o godzinie 12.00 zostanie zarządzona półgodzinna przerwa na otwarcie wystawy „pierwsze/najstarsze”, poświęconej pierwszym tekstom i najstarszym zabytkom kultury i historii dokumentującym początki państwa polskiego. informuję również państwa, że do rozpatrzenia punktu pierwszego przystąpimy dziś po przerwie na wystawę, czyli po godzinie 12.30. tak więc do rozpatrzenia punktu pierwszego przystąpimy później. a w tej chwili przystępujemy do rozpatrzenia punktu drugiego porządku obrad… czy jest głos sprzeciwu w sprawie rozpatrzenia punktu pierwszego po przerwie? pan marszałek borusewicz jest przeciw. tak?",
37397,"informacja o wpływie interpelacji i zapytań oraz odpowiedzi na nie dostępna jest w systemie informacyjnym sejmu. na tym kończymy 37. posiedzenie sejmu rzeczypospolitej polskiej. protokół posiedzenia będzie wyłożony do przejrzenia w sekretariacie posiedzeń sejmu. porządek dzienny 38. posiedzenia sejmu, zwołanego na dni 29, 30 września i 1 października 2021 r., został paniom i panom posłom doręczony. zamykam posiedzenie.",
1614028,"informuję, że protokół siódmego posiedzenia senatu rzeczypospolitej polskiej ósmej kadencji zostanie udostępniony senatorom w terminie trzydziestu dni po posiedzeniu senatu w biurze prac senackich, pokój nr 255. zamykam siódme posiedzenie senatu rzeczypospolitej polskiej ósmej kadencji.",
635146,"informacja o wpływie interpelacji, zapytań oraz odpowiedzi na nie została paniom i panom posłom doręczona w druku nr 3906. na tym kończymy 86. posiedzenie sejmu rzeczypospolitej polskiej. protokół posiedzenia będzie wyłożony do przejrzenia w sekretariacie posiedzeń sejmu. porządek dzienny 87. posiedzenia sejmu, zwołanego na dni 16, 17 i 18 marca 2011 r., został paniom i panom posłom doręczony. zamykam posiedzenie.",
716880,"dziękuję, panie ministrze. zamykam dyskusję. do głosowania nad uchwałą senatu przystąpimy w bloku głosowań. ogłaszam przerwę do godz. 20. o godz. 20 rozpoczną się głosowania.",
1631758,"dziękuję bardzo. proszę o komunikaty. a, nie ma komunikatów. informuję, że protokół trzydziestego siódmego posiedzenia senatu rzeczypospolitej polskiej ósmej kadencji zostanie udostępniony senatorom w terminie trzydziestu dni po posiedzeniu senatu w biurze prac senackich, pokój nr 255. zamykam trzydzieste siódme posiedzenie senatu rzeczypospolitej polskiej ósmej kadencji.",
42323,"dziękuję, panie pośle. kontynuujemy rozpatrywanie punktu 15. porządku dziennego. pan poseł krzysztof grabczuk, koalicja obywatelska. pan poseł krzysztof grabczuk, koalicja obywatelska. bardzo proszę. będziemy głosować w bloku głosowań, panie pośle. ale proszę nie krzyczeć. bardzo proszę, panie pośle, bo czas upłynie. panie pośle, czas.",
1616456,"dziękuję, panie senatorze. więcej oświadczeń nie ma. informuję państwa, że protokół dwunastego posiedzenia senatu rzeczypospolitej polskiej ósmej kadencji zostanie udostępniony senatorom w terminie trzydziestu dni po posiedzeniu senatu w biurze prac senackich, pokój nr 255. zamykam dwunaste posiedzenie senatu rzeczypospolitej polskiej ósmej kadencji.",
1633129,"dziękuję bardzo, panie senatorze. stwierdzam, że lista mówców została wyczerpana. informuję państwa senatorów, że protokół trzydziestego ósmego posiedzenia senatu rzeczypospolitej polskiej ósmej kadencji zostanie udostępniony senatorom w terminie trzydziestu dni po posiedzeniu senatu w biurze prac senackich, pokój nr 255. zamykam trzydzieste ósme posiedzenie senatu rzeczypospolitej polskiej ósmej kadencji.",
1644610,"bardzo dziękuję, panie senatorze. nie ma więcej chętnych. informuję, że protokół sześćdziesiątego pierwszego posiedzenia senatu rzeczypospolitej polskiej ósmej kadencji zostanie udostępniony senatorom w termin

do wyrzucenia

## Topic 11

In [53]:
sejm_t11 = sejm[sejm['detected_topic'] == 11]
random_sample = sejm_t11.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
58953                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              panie marszałku! wysoka izbo! panie ministrze! dobrze, że rząd znalazł dodatkowe środki na rozwój dróg lokalnych. dobrze, że rząd przedłuża program finansowania inwestycji drogowych o kolejne 2 lata. cechą wspólną kilku programów takich jak ˝narodowy program przebudowy dróg lokalnych˝, który realizowany był bardzo skutecznie w latach 2008-2011, 2012-2015, a także już w czasie rządu prawa i sprawiedliwości w latach 2016-2019 czy też funduszu dróg samorządowych, ustawa została uchwalona w 2018 r., zmieniony w rządowy fundusz rozwoju dróg w listopadzie 2020 r. - cechą wspólną tych wszystkich programów jest wsparcie inwestycji samorządowych. to łączy działania tego rządu i rządów poprzednich. natomiast projekt ustawy przewiduje niebezpieczne rozwiązanie. to rozwiązanie, które przenosi fundusz celowy, w zasadzie likwiduje fundusz celowy i przekształca go jedynie w rachunek w banku gospodarstwa krajowego, ograniczy kontrolę parlamentu, ograniczy kontrolę sejmu nad wydatkami z tego funduszu. i w tej chwili zapowiadam, że jeżeli ta zmiana zostanie dokonana, my wykorzystując ustawę o wykonywaniu mandatu posła i senatora, będziemy skutecznie realizowali kontrole poselskie w tym zakresie. dlaczego? dlatego że samorządy skarżą się, iż są nierówno traktowane, dlatego że do tej pory była taka praktyka, mimo iż parlament mógł to kontrolować i co roku przyjmowaliśmy sprawozdanie z realizacji rządowego funduszu rozwoju dróg, także i w tym roku przyjmowaliśmy to sprawozdanie, ale samorządy skarżyły się, że są nierówno traktowane, że są nieczytelne kryteria. potwierdza to także rozwiązanie, które było i które jest utrzymywane jest w tej ustawie: daje ono prezesowi rady ministrów po otrzymaniu listy od ministra właściwego do spraw transportu możliwość dokonywania zmian. to nie tylko dokonywanie zmian, ale wprowadzanie na listę zadań, które wcześniej nie były zgłoszone w wyniku procedury, któr

In [54]:
sejm_t11['topic_name'] = 'infrastruktura drogowa'
sejm.loc[sejm.index.isin(sejm_t11.index), 'topic_name'] = sejm_t11['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/2974800257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t11['topic_name'] = 'infrastruktura drogowa'


## Topic 7

In [55]:
sejm_t7 = sejm[sejm['detected_topic'] == 7]
random_sample = sejm_t7.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

,text_all,topic_name
680296,"co więcej, muszę przyznać, że nie tylko premier nawoływał do tego, aby nie płacić abonamentu, i do tego, aby łamać prawo. otóż biskup włocławski wiesław mering w katedrze wezwał z ambony do niepłacenia abonamentu. nawiązywał również do łamania prawa, stawiając warunek, aby telewizja polska pozbyła się nergala.",
25814,"panie marszałku! szanowny panie ministrze! w czasie dzisiejszej debaty dotyczącej bezpieczeństwa w ruchu drogowym chciałbym zapytać, czy w resorcie trwają jakieś prace na temat tzw. trzeciego sygnału dla pojazdów uprzywilejowanych. chodzi o to, że w dzisiejszych rozwiązaniach ta sygnalizacja jest niewystarczająca i to rozwiązanie powodowałoby zwiększenie tras życia, m.in. poprzez sygnały dźwiękowe, ale także poprzez radio. wtedy w jadącym pojeździe, mającym włączone radio, to radio by się wyłączało i włączałyby się komunikaty dotyczące właśnie pojazdów uprzywilejowanych. z danych państwowej straży pożarnej wynika, że w ostatnich 7 latach pojazdy uprzywilejowane uczestniczyły w 700 wypadkach i kolizjach i stąd potrzeba podjęcia prac strategicznych związanych (dzwonek) ewentualnie z rozważeniem wdrożenia tego rozwiązania.",
802531,"panie marszałku! pani premier! wysoka izbo! pewnie jak wielu jestem również pod wrażeniem szczerości wypowiedzi marszałka terleckiego, który - cytuję tutaj za mediami - powiedział dzisiaj: jeżeli media wyobrażają sobie, że będą zajmować polaków krytykowaniem zmian, to trzeba to przerwać. dziwi mnie ta wypowiedź, bo jeśli mnie pamięć nie myli, pan marszałek terlecki w czasach minionego systemu, tzw. komuny, był tą osobą, która wolne media tworzyła, był wśród tych, co w wolnej polsce zakładali pierwszy niezależny dziennik - ˝czas krakowski˝, był tą osobą, która zachęcała do dziennikarstwa wielu młodych ludzi, w tym również mojego brata. dzisiaj pan marszałek jest twarzą zamachu na jedną z podstawowych wolności - wolność słowa i niezależne media. aż nie chce mi się wierzyć, że forsujecie państwo swego rodzaju komisję weryfikacyjną w osobie ministra skarbu państwa, która będzie dowodzić czystką w zarządach publicznej telewizji i radia. czy naprawdę pan marszałek chce być zapamiętany jako grabarz wolnych mediów? dziękuję bardzo.",
585754,"nawiasem mówiąc, nie dziwię się temu, że pan o to pyta, bo faktem jest, że ten komunikat został podany do wiadomości publicznej, ale żadne z tzw. mainstreamowych mediów go nie zamieściło. szukałem, czy jest w ˝trybunie˝, też nie było. co do pani jarugi-nowackiej to w kwestii, czy następuje tutaj pogłębianie różnic, czy deptane są prawa innych wyznań - stanowisko episkopatu już jest znane - czy nie można by wprowadzić jakiegokolwiek dnia wolnego i niech każdy sobie wybiera kiedy - to jest to propozycja, która - nie sądzę, żeby pani wiedziała o tym wtedy, kiedy pani taką propozycję wysuwała - w związku sowieckim nazywała się wychodnoj dien. chodziło dokładnie o to, żeby zlikwidować świętowanie świąt chrześcijańskich w określone dni, wobec tego zamiast niedzieli - wychodnoj dien, bo tam chcieli sięgnąć tak daleko.",
1155684,"większość tych zastrzeżeń, które pani dyrektor przedstawiła, albo już została zrealizowana, albo jest realizowana pod nadzorem biura prawnego msz. najbardziej, nazwijmy to, kontrowersyjny punkt dotyczy tej audycji telewizyjnej, ale to po prostu wynika ze specyfiki działania mediów elektronicznych, dlatego że mówimy tutaj o cyklu czterech audycji telewizyjnych, które zostały zaplanowane w budżecie na 2022 r. faktury zostały wystawione w 2022 r., tak jak to było w dokumentach przedstawionych najwyższej izbie kontroli. natomiast cykl produkcyjny tego rodzaju audycji nie jest kilkudniowy czy kilkutygodniowy, w związku z tym ta audycja została wyemitowana na początku 2023 r. dziękuję bardzo.",
1708843,"panie przewodniczący, ja mam trzy pytania. pierwsze pytanie. no, już kilka miesięcy temu toczyła się dyskusja o finansowaniu mediów publicznych. czy spółka prawa handlowego jest najlepszą formą prawno-organizacyjną za

In [56]:
sejm_t7['topic_name'] = 'media publiczne'
sejm.loc[sejm.index.isin(sejm_t7.index), 'topic_name'] = sejm_t7['topic_name'].values
sejm.to_csv("data/sejm_bertopic_updated.csv")

/var/folders/bm/ftw8z7yx4x7__tj58fqqzvcm0000gn/T/ipykernel_26558/2232967863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sejm_t7['topic_name'] = 'media publiczne'


## Topic 38

In [9]:
sejm_t38 = sejm[sejm['detected_topic'] == 38]
random_sample = sejm_t38.sample(n=20, random_state=42)
random_sample[['text_all', 'topic_name']]

text_all  \
200511                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 panie marszałku! pani minister! wysoka izbo! mam zaszczyt przedstawić stanowisko klubu kukiz’15 w sprawie projektu ustawy o zmianie ustawy o wspieraniu rozwoju usług i sieci telekomunikacyjnych oraz niektórych innych ustaw. omawiana ustawa, jak zaznaczono w uzasadnieniu, ma na celu m.in. implementację dyrektywy parlamentu europejskiego i rady dotyczącej zmniejszenia kosztów realizacji szybkich sieci łączności elektronicznej. ma ona również na celu usunięcie barier administracyjnych i prawnych, które często blokują inwestycje telekomunikacyjne. szanowni państwo! wszyscy zgodzimy się, że rozwój sieci teleinformatycznych jest sprawą kluczową dla każdego państwa stawiającego na rozwój. sieć teleinformatyczna to nie tylko - jak wybrzmiało z tej mównicy - internet, ale to także sieci komórkowe i innego rodzaju sieci, które zostały wyłączone z infrastruktury krytycznej, jak sieci nadzoru nad często kluczowymi elementami infrastruktury. oczywiście jeżeli chodzi o internet, dla użytkownika prywatnego jest to znacząca pomoc w wielu dziedzinach życia i powinniśmy dążyć do zapewnienia jak najłatwiejszego i jak najtańszego dostępu do sieci wszystkim mieszkańcom naszego kraju. omawiana nowelizacja wprowadza m.in. zmiany powodujące rozszerzenie kręgu podmiotów dysponujących infrastrukturą techniczną, zobowiązanych do jej udostępniania przedsiębiorcom telekomunikacyjnym, którzy wykorzystają ją z przeznaczeniem na budowę szybkich sieci telekomunikacyjnych. kolejna zmiana, która ma na celu sprawniejszą rozbudowę, jak również budowę nowej infrastruktury sieciowej, polega na nałożeniu na firmy budowlane obowiązku wyposażenia budynków mieszkalnych wielorodzinnych, jak również użyteczności publicznej w instalację telekomunikacyjną, jeżeli nie są one w nią wyposażone, a są poddawane rozbudowie, przebudowie lub nadbudowie. zmiany dotyczą również koordynacji robót budowlanych w taki sposób, żeby firma telekomunikacyjna mogła rozpocząć budowę sieci w tym samym miejscu, w którym rozpoczął ją inny podmiot 

In [ ]:
sejm_t38 = sejm_t38.drop("Unnamed: 0", axis=1)

In [28]:

sejm_t38.reset_index(inplace=True)
sejm_t38.head(0)

,index,date,author,text_all,place,TextBlob_Subjectivity,TextBlob_Polarity,sentiment_herbert,openai_result,text_part,sentiment,topic,is_outlier,detected_topic,topic_name


In [29]:

def bertopic_guided(df, topic_list, n_topics = 50):
    representation_model = KeyBERTInspired(random_state=42)
    calculate_probabilities = True # get document's topic probs for each detected topic, not just the predicted one
    umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
    nr_topics: int | str = n_topics
    topic_model = BERTopic(language='polish', nr_topics=nr_topics, verbose=True, representation_model=representation_model, calculate_probabilities=calculate_probabilities, umap_model=umap_model, seed_topic_list=topic_list)
    
    df.text_all = df.text_all.astype(str)

    topics, probs = topic_model.fit_transform(df.text_all)

    topic_model.visualize_barchart(top_n_topics=nr_topics if isinstance(nr_topics, int) else 50, n_words=20)
    
    return topic_model, topics, probs

In [30]:
seed_topic_list = [ ["podatki", "podatek", "opodatkowanie", "podatkowe"],
]

topic_model, topics, probs = bertopic_guided(sejm_t38, seed_topic_list, 3)

Batches: 100%|██████████| 14/14 [00:06<00:00,  2.05it/s]
2023-12-31 04:40:52,901 - BERTopic - Transformed documents to Embeddings
Batches: 100%|██████████| 1/1 [00:00<00:00, 68.39it/s]
2023-12-31 04:40:56,492 - BERTopic - Reduced dimensionality
2023-12-31 04:40:56,505 - BERTopic - Clustered reduced embeddings
2023-12-31 04:40:58,228 - BERTopic - Reduced number of topics from 4 to 3


In [31]:
ignore_outlier = True
topic_info_df = topic_model.get_topic_info()
if (ignore_outlier):
    topic_info_df = topic_info_df.loc[topic_info_df.Topic != -1]

fig = px.histogram(y=topic_info_df['Name'], x=topic_info_df['Count'], orientation='h', title='Histogram of topic occurences' + (' (outlier topic -1 omitted)' if ignore_outlier else ''))
fig.update_layout(height=900)
fig.update_xaxes(title_text='# of Occurences')

# Update y-axis label
fig.update_yaxes(title_text='Topic name')
fig.show()

In [36]:
new_topics = topic_model.reduce_outliers(sejm_t38.text_all, topics, probabilities=probs, strategy='probabilities')

In [37]:
sejm_t38['detected_topic'] = new_topics

In [48]:
sejm_t38['detected_topic'].unique()

array([0, 1, 2])

In [49]:
topic_model.visualize_barchart(top_n_topics=3 if isinstance(3, int) else 50, n_words=20)

Po podziale guided wyszczególniono temat podatki

In [50]:
topic_mapping = {0: 'podatki', 1: '', 2: ''}
sejm_t38['topic_name'] = sejm_t38['detected_topic'].map(topic_mapping)

In [51]:
sejm_t38['topic_name'].unique()

array(['podatki', ''], dtype=object)

In [52]:
for index, row in sejm_t38.iterrows():
    text_all = row['text_all']
    topic_name_5 = row['topic_name']

    sejm.loc[sejm['text_all'] == text_all, 'topic_name'] = topic_name_5

In [55]:
sejm.to_csv("data/sejm_bertopic_updated.csv")

## Topic 18

In [57]:
sejm_t18 = sejm[sejm['detected_topic'] == 18]
random_sample = sejm_t18.sample(n=5, random_state=42)
random_sample[['text_all', 'topic_name']]

,text_all,topic_name
135689,"nie twierdzę, że jest to doskonały projekt. nie stwierdzę, że jest on doskonały. natomiast wychodzę z założenia, że da się to racjonalnie poprawić i z tej skrajności, którą zaproponował ruch palikota, z tych trzech projektów jesteśmy w stanie zaproponować wysokiej izbie coś sensownego. coś trzeba zaproponować, bo podstawowe wartości dotyczące czci i godności człowieka w zakresie jego orientacji seksualnej, w zakresie jego skłonności, w zakresie jego tożsamości płciowej powinny podlegać ochronie. i to jest poza sporem. na to przymykać oczu nie wolno - bo do tej pory kodeks karny na ten temat milczał - i w tym zakresie chyba się godzimy.",NaN
21941,"dziękuję. szanowny panie marszałku! wysoka izbo! szanowny panie ministrze! 4 listopada 2016 r. sejm uchwalił ustawę o wsparciu kobiet w ciąży i rodzin „za życiem”. celem ustawy było uregulowanie po raz pierwszy w sposób kompleksowy - chcę podkreślić to słowo: kompleksowy - wsparcia dla kobiet w ciąży i rodzin, ze szczególnym uwzględnieniem kobiet w ciąży powikłanej oraz w sytuacji niepowodzeń położniczych, a także dla dzieci, u których zdiagnozowano ciężkie i nieodwracalne upośledzenie albo nieuleczalną chorobę zagrażającą ich życiu, które powstały w prenatalnym okresie rozwoju dziecka lub w czasie porodu. na podstawie tej ustawy opracowany został program kompleksowego wsparcia dla rodzin „za życiem” przyjęty uchwałą rady ministrów z dnia 20 grudnia 2016 r. tak jak powiedziałam, było to pierwsze kompleksowe wsparcie. było ono realizowane nie tylko przez ministerstwo rodziny i polityki społecznej. obejmowało też opiekę zdrowotną i pomoc finansową. było to rozwiązanie, wydawało się na tamten moment, bardzo nowatorskie, nowe, tak jak powiedziałam, kompleksowe. w pytaniu tym chcielibyśmy właśnie zapytać o podsumowanie funkcjonowania programu „za życiem”, dotychczasowego funkcjonowania. jakie elementy programu dotychczas można uznać za najbardziej służące wsparciu kobiet w okresie ciąży, porodu i połogu, ze szczególnym uwzględnieniem, jeszcze raz podkreślę, kobiet w ciąży powikłanej, oraz wsparciu ich rodzin, a także wczesnemu wspomaganiu rozwoju dziecka? ile funduszy przeznaczono dotąd z budżetu państwa (dzwonek) na realizację programu „za życiem”? teraz została ogłoszona nowa edycja, widać więc, że ten program potrzebuje jednak uzupełnienia. dlatego chcielibyśmy zapytać też, na czym będzie polegała nowa edycja programu „za życiem”.",NaN
94354,"panie marszałku! wysoka izbo! moje pytania dotyczą rozdziału zatytułowanego ˝niania˝. pytanie pierwsze. czy nianią, w myśl tych zapisów, może być tylko kobieta? czy mężczyzna też może pełnić tę funkcję? jeżeli tak, to jak się będzie nazywał? ˝niań˝, ˝niańczyk˝ czy jeszcze jakoś inaczej? to po pierwsze.",NaN
240392,"panie marszałku! wysoka izbo! szanowni państwo! przedstawiony projekt ustawy powinien się nazywać ˝zniszczyć polskie kobiety˝, a pani nowacka w całym uzasadnieniu zaprezentowała brak szacunku do kobiet. o godności w ogóle pani zapomniała.",NaN
240615,"panie marszałku! wysoka izbo! jesteśmy przeciwko dręczeniu kobiet, dlatego platforma obywatelska złożyła wniosek o odrzucenie projektu, który zaostrza przepisy aborcyjne. nie ma i nie będzie naszej zgody na zmuszanie kobiet do cierpienia ani na ich ubezwłasnowolnienie. kobieta musi mieć prawo wyboru - prawo wyboru, które jest zagwarantowane w kompromisie aborcyjnym, a które to prawo pis chce kobietom dzisiaj zabrać, topiąc się w hipokryzji, topiąc się w ideologii, pogardzie dla kobiet i na końcu - wytycznych posła kaczyńskiego. dziękuję.",NaN


In [61]:
sejm_t18.reset_index(inplace=True)

In [62]:
seed_topic_list = [ ["aborcja", "aborcji", "aborcję", "ciąży", "przerywanie"],
                   ["kobiety", "kobiet", "prawa"],
                   ["lgbt", "homoseksualy"]
]

topic_model, topics, probs = bertopic_guided(sejm_t18, seed_topic_list, 4)

Batches: 100%|██████████| 30/30 [00:12<00:00,  2.36it/s]
2023-12-31 14:50:17,917 - BERTopic - Transformed documents to Embeddings
Batches: 100%|██████████| 1/1 [00:00<00:00, 44.68it/s]
2023-12-31 14:50:21,104 - BERTopic - Reduced dimensionality
2023-12-31 14:50:21,140 - BERTopic - Clustered reduced embeddings
2023-12-31 14:50:21,889 - BERTopic - Reduced number of topics from 3 to 3


In [63]:
ignore_outlier = True
topic_info_df = topic_model.get_topic_info()
if (ignore_outlier):
    topic_info_df = topic_info_df.loc[topic_info_df.Topic != -1]

fig = px.histogram(y=topic_info_df['Name'], x=topic_info_df['Count'], orientation='h', title='Histogram of topic occurences' + (' (outlier topic -1 omitted)' if ignore_outlier else ''))
fig.update_layout(height=900)
fig.update_xaxes(title_text='# of Occurences')

# Update y-axis label
fig.update_yaxes(title_text='Topic name')
fig.show()

In [64]:
new_topics = topic_model.reduce_outliers(sejm_t18.text_all, topics, probabilities=probs, strategy='probabilities')

In [65]:
sejm_t18['detected_topic'] = new_topics

In [66]:
topic_model.visualize_barchart(top_n_topics=3 if isinstance(3, int) else 50, n_words=20)

In [67]:
topic_mapping = {0: 'aborcja', 1: 'społeczność lgbt', 2: 'prawa kobiet'}
sejm_t18['topic_name'] = sejm_t18['detected_topic'].map(topic_mapping)

In [68]:
for index, row in sejm_t18.iterrows():
    text_all = row['text_all']
    topic_name_5 = row['topic_name']

    sejm.loc[sejm['text_all'] == text_all, 'topic_name'] = topic_name_5

In [ ]:
sejm.to_csv("data/sejm_bertopic_updated.csv")

## Topic 2

In [69]:
sejm_t2 = sejm[sejm['detected_topic'] == 2]

In [87]:
embedding_model='paraphrase-multilingual-MiniLM-L12-v2'
sentence_model = SentenceTransformer(embedding_model, device='cpu')
embeddings: np.ndarray = sentence_model.encode(sejm_t2.text_all.to_numpy(), show_progress_bar=True, batch_size=128)
representation_model = KeyBERTInspired(random_state=42)
calculate_probabilities = True # get document's topic probs for each detected topic, not just the predicted one
umap_model = UMAP(n_neighbors=30, n_components=5, min_dist=0.0, metric='cosine', random_state=42, verbose=True)
nr_topics: int | str = 20
topic_model = BERTopic(embedding_model=embedding_model, nr_topics=nr_topics, verbose=True, representation_model=representation_model, calculate_probabilities=calculate_probabilities, umap_model=umap_model, low_memory=True)
topics, probs = topic_model.fit_transform(sejm_t2.text_all, embeddings=embeddings)


Batches: 100%|██████████| 80/80 [02:06<00:00,  1.58s/it]


UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_components=5, n_neighbors=30, random_state=42, verbose=True)
Sun Dec 31 16:58:12 2023 Construct fuzzy simplicial set
Sun Dec 31 16:58:12 2023 Finding Nearest Neighbors
Sun Dec 31 16:58:12 2023 Building RP forest with 10 trees
Sun Dec 31 16:58:12 2023 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	Stopping threshold met -- exiting after 4 iterations
Sun Dec 31 16:58:13 2023 Finished Nearest Neighbor Search
Sun Dec 31 16:58:13 2023 Construct embedding


Epochs completed:   6%| ▌          11/200 [00:00]

	completed  0  /  200 epochs


Epochs completed:  14%| █▎         27/200 [00:01]

...

Epochs completed:  96%| █████████▌ 191/200 [00:05]

	completed  180  /  200 epochs


Epochs completed: 100%| ██████████ 200/200 [00:06]
2023-12-31 16:58:19,615 - BERTopic - Reduced dimensionality


Sun Dec 31 16:58:19 2023 Finished embedding


2023-12-31 16:58:20,485 - BERTopic - Clustered reduced embeddings
2023-12-31 16:58:32,229 - BERTopic - Reduced number of topics from 36 to 20


In [88]:
topic_model.visualize_barchart(top_n_topics=nr_topics if isinstance(nr_topics, int) else 50, n_words=20)

In [83]:
ignore_outlier = True
topic_info_df = topic_model.get_topic_info()
if (ignore_outlier):
    topic_info_df = topic_info_df.loc[topic_info_df.Topic != -1]

fig = px.histogram(y=topic_info_df['Name'], x=topic_info_df['Count'], orientation='h', title='Histogram of topic occurences' + (' (outlier topic -1 omitted)' if ignore_outlier else ''))
fig.update_layout(height=900)
fig.update_xaxes(title_text='# of Occurences')

# Update y-axis label
fig.update_yaxes(title_text='Topic name')
fig.show()

In [90]:
new_topics = topic_model.reduce_outliers(sejm_t2.text_all, topics, probabilities=probs, strategy='probabilities')

In [91]:
sejm_t2['detected_topic'] = new_topics

In [92]:
topic_mapping = {0: 'prawa dziecka', 
                 1: 'publiczna opieka zdrowotna', 
                 2: 'publiczna opieka zdrowotna',
                 3: 'publiczna opieka zdrowotna',
                 4: 'publiczna opieka zdrowotna',
                 5: 'szczepienia',
                 7: '',
                 8: 'publiczna opieka zdrowotna',
                 9: 'publiczna opieka zdrowotna',
                 10: 'publiczna opieka zdrowotna',
                 11: 'publiczna opieka zdrowotna',
                 12: 'publiczna opieka zdrowotna',
                 13: '',
                 14: '', 
                 15: 'publiczna opieka zdrowotna',
                 16: 'publiczna opieka zdrowotna',
                 17: 'publiczna opieka zdrowotna',
                 18: 'publiczna opieka zdrowotna'}
sejm_t2['topic_name'] = sejm_t2['detected_topic'].map(topic_mapping)

In [93]:
sejm_t2['topic_name'].unique()

array(['szczepienia', 'prawa dziecka', 'publiczna opieka zdrowotna', '',
       nan], dtype=object)

In [94]:
for index, row in sejm_t2.iterrows():
    text_all = row['text_all']
    topic_name_5 = row['topic_name']

    sejm.loc[sejm['text_all'] == text_all, 'topic_name'] = topic_name_5

In [95]:
sejm.to_csv("data/sejm_bertopic_updated.csv")

In [96]:
s = pd.read_csv("data/sejm_bertopic_updated.csv")


In [97]:
s['topic_name'].unique()

array([nan, 'szczepienia', 'infrastruktura drogowa', 'prawa dziecka',
       'publiczna opieka zdrowotna', 'podatki', 'kontrola PIP i sanepidu',
       'aborcja', 'media publiczne', 'prawa kobiet',
       'zaostrzenie przepisów dotyczących pedofilii', 'wyścigi konne',
       'społeczność lgbt', 'zakaz handlu w niedzielę',
       'prawo antyterrorystyczne', 'pomoc dla Ukrainy', 'prawo budowlane'],
      dtype=object)